In [3]:
from dataclasses import dataclass
from transformers import PreTrainedModel, PreTrainedTokenizerFast, PretrainedConfig
from composer.models.huggingface import HuggingFaceModel

from xformers.components.feedforward import FusedMLP
from xformers.triton import FusedLayerNorm

import torch
import torch.nn as nn
import torch.nn.functional as F


In [5]:
@dataclass
# need to fix this config
class WabiSabiConfig(PretrainedConfig):
    """
    Important ratios:
    - d_model should be a multiple of n_heads
    - d_q, d_k, d_v are all equal to d_model / n_heads
    """

    d_model: int = 2048
    n_heads: int = 16
    n_layers: int = 24
    vocab_size: int = 50368

    def __post_init__(self):
        self.d_head = self.d_model // self.n_heads


class WSMultiQueryAttention(nn.Module):
    def __init__(self, d_model: int, d_head: int):
        super().__init__()

        # NOTE: this layer concatenates several tensors together
        # When initializing params, we should instantiate these separately.
        # output size: q -> d_model, k and v -> head_dim
        self.residual_to_qkv = nn.Linear(d_model, d_model + 2 * d_head)

        self.concat_attention_to_residual = nn.Linear(d_model, d_model)
        self.concat_attention_to_residual._is_residual_projection = True

    def forward(self, x: torch.Tensor):
        qkv = self.residual_to_qkv(x)
        # split qkv into q, k, v
        q, k, v = torch.split(qkv, [self.d_model, self.d_head, self.d_head], dim=-1)

        # PyTorch's flash attention implementation
        attention = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        x = self.concat_attention_to_residual(attention)
        return x


# Flash attention, multi-query attention
class WSBlock(nn.Module):
    def __init__(
        self,
        d_model: int,
        d_head: int,
    ):
        super().__init__()
        self.layer_norm_before_attention = FusedLayerNorm(d_model)
        self.attention = WSMultiQueryAttention(d_model, d_head)
        self.layer_norm_before_ffn = FusedLayerNorm(d_model)

        # this ratio is standard for transformers
        # Triton fused MLP. The linear layers in PyTorch are already fused,
        # but xformers has a custom implementation that fuses dropout and bias
        ffn_ratio = 4
        self.ffn = FusedMLP(
            dim_model=d_model,
            dropout=0.0,
            activation="gelu",
            hidden_layer_multiplier=ffn_ratio,
        )
        # index of the last linear layer
        self.ffn.mlp[2]._is_residual_projection = True

    def forward(self, x: torch.Tensor):
        x_norm = self.layer_norm_before_attention(x)
        x_attn = self.attention(x_norm)
        x = x + x_attn

        x_norm = self.layer_norm_before_ffn(x)
        x_ffn = self.ffn(x_norm)
        x = x + x_ffn
        return x


class WabiSabiModel(PreTrainedModel):
    def __init__(self, config: WabiSabiConfig):
        super().__init__(config)
        self.config = config
        self.tokens_to_embeddings = nn.Embedding(
            num_embeddings=config.vocab_size, embedding_dim=config.d_model
        )

        # Embedding fraction: page 7 of GLM-130B paper https://arxiv.org/abs/2210.02414
        # self.embedding_fraction = config.embedding_fraction

        self.blocks = nn.ModuleList(
            [
                WSBlock(
                    d_model=config.d_model,
                    d_head=config.d_head,
                )
                for _ in range(config.n_layers)
            ]
        )

        # Could also consider the Mosaic implementation...
        # https://docs.mosaicml.com/projects/composer/en/latest/method_cards/low_precision_layernorm.html
        self.layer_norm_final = FusedLayerNorm(config.d_model)

        self.embeddings_to_logits = nn.Linear(
            in_features=config.d_model, out_features=config.vocab_size
        )

        # https://paperswithcode.com/method/weight-tying
        self.embeddings_to_logits.weight = self.tokens_to_embeddings.weight

        # initialize parameters
        # notes from MPT/nanoGPT/transformers
        # 1. residual projections (e.g. linear layers that project to d_model) are divided
        # by 1 / sqrt(num_layers)
        # 2. layer norm weights are set to one (PyTorch sets this by default; skip)
        # 3. all others are initialized with normal distribution with mean 0 and std 0.02
        # Note: MPT uses kaiming_normal; I'll go for this as well
        def init_weights(module: nn.Module):
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module._is_residual_projection:
                    with torch.no_grad():
                        module.weight.div_(torch.sqrt(config.n_layers))

            elif isinstance(module, nn.Embedding):
                nn.init.kaiming_normal_(module.weight)

        # disable bias in all modules
        # note for later: if you want to enable bias, should remember to zero out all biases
        # in init_weights
        def disable_bias(module: nn.Module):
            if hasattr(module, "bias") and isinstance(module.bias, nn.Parameter):
                module.register_parameter("bias", None)

        self.apply(init_weights)
        self.apply(disable_bias)

    # TODO: kwargs are for other HuggingFace generate params. Implement if needed.
    def forward(self, input_ids: torch.LongTensor, **kwargs):
        x = self.tokens_to_embeddings(input_ids)

        # MPT doesn't use embedding fraction
        # x = (x * self.embedding_fraction) + (x.detach() * (1 - self.embedding_fraction))

        for block in self.blocks:
            x = block(x)

        x = self.layer_norm_final(x)
        x = self.embeddings_to_logits(x)
        return x

In [6]:
model = WabiSabiModel(WabiSabiConfig())


AttributeError: cannot assign module before Module.__init__() call